In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(12_000_000_000_000)

import pandas as pd
from siuba import *

from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.tables import tbls
import shared_utils
import datetime as dt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/cloud/bigquery/table.py:43: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/

In [57]:
from segment_speed_utils.project_vars import (PREDICTIONS_GCS, 
                                              analysis_date)

In [58]:
PREDICTIONS_GCS

'gs://calitp-analytics-data/data-analyses/rt_predictions/'

In [59]:
analysis_date = dt.datetime.fromisoformat(analysis_date)
analysis_date

datetime.datetime(2023, 3, 15, 0, 0)

# Sandbox

In [5]:
am_sampling_period = [dt.datetime.combine(analysis_date, dt.time(8, 0)),
                     dt.datetime.combine(analysis_date, dt.time(8, 30))]
mid_sampling_period = [dt.datetime.combine(analysis_date, dt.time(12, 30)),
                     dt.datetime.combine(analysis_date, dt.time(13, 0))]
pm_sampling_period = [dt.datetime.combine(analysis_date, dt.time(23, 0)),
                     dt.datetime.combine(analysis_date, dt.time(23, 30))]

In [6]:
sampling_periods = [am_sampling_period, mid_sampling_period, pm_sampling_period]

In [7]:
bbb_base64 = 'aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZGF0ZXMuYmlu'

In [20]:
metro_bus_base64 = 'aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbGFtZXRyby9ndGZzLXJ0LXRyaXAtdXBkYXRlcw=='

In [21]:
def get_sample_df(sampling_period):
    df = (
    tbls.mart_ad_hoc.fct_stop_time_updates_20230315_to_20230321()
    >> filter(_.service_date == analysis_date,
              # _.trip_id == '894836',
              _.arrival_time_pacific >= sampling_period[0],
              _.arrival_time_pacific < sampling_period[1],
             _.base64_url == metro_bus_base64)
    >> select(_.arrival_time_pacific, _.departure_time_pacific,
             _.key, _.gtfs_dataset_key, _.base64_url,
              _._extract_ts, _.trip_update_timestamp, _.trip_id,
             _.stop_sequence, _.stop_id, _.service_date)
    >> collect()
)
    return df

In [22]:
metro_am_sample = get_sample_df(am_sampling_period)

In [23]:
metro_am_sample

,arrival_time_pacific,departure_time_pacific,key,gtfs_dataset_key,base64_url,_extract_ts,trip_update_timestamp,trip_id,stop_sequence,stop_id,service_date
0,2023-03-15 08:09:01,NaT,ab6a1a23a8d65b3cef36aea5cf6e56d9,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:58:00+00:00,2023-03-15 14:57:40+00:00,10040009450628-DEC22,78,10906,2023-03-15
1,2023-03-15 08:09:49,NaT,b442788d9d72826e90b2082a332dd120,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 15:09:40+00:00,2023-03-15 15:09:16+00:00,10040009450628-DEC22,80,10908,2023-03-15
2,2023-03-15 08:07:28,NaT,e3b55990425617c2616ab598b2c56acc,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:56:20+00:00,2023-03-15 14:56:00+00:00,10040009450628-DEC22,77,10905,2023-03-15
3,2023-03-15 08:04:59,NaT,e9c84f5b8a43d32737635da4bfad775c,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:20:20+00:00,2023-03-15 14:20:00+00:00,10040009450628-DEC22,78,10906,2023-03-15
4,2023-03-15 08:07:53,NaT,b22e3457116bc789b9ba3f2a194408a7,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:39:20+00:00,2023-03-15 14:39:00+00:00,10040009450628-DEC22,80,10908,2023-03-15
...,...,...,...,...,...,...,...,...,...,...,...
3912041,2023-03-15 08:17:11,NaT,fd8f11801186dffea5554e909f94d5a4,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 15:14:20+00:00,2023-03-15 15:14:07+00:00,10081003230646-DEC22,76,13457,2023-03-15
3912042,2023-03-15 08:12:33,NaT,6b1973bf672d30658bc25b45710341b0,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:48:40+00:00,2023-03-15 14:48:20+00:00,10081003230646-DEC22,76,13457,2023-03-15
3912043,2023-03-15 08:10:46,NaT,80b411ee9836bf74dd4ab569d3b5cb82,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:39:00+00:00,2023-03-15 14:38:38+00:00,10081003230646-DEC22,76,13457,2023-03-15
3912044,2023-03-15 08:12:56,NaT,586d7443aa1ac1e0cfbc2f551df8cd54,a163bb0928257e3654888c511f4b56dc,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,2023-03-15 14:52:00+00:00,2023-03-15 14:51:49+00:00,10081003230646-DEC22,76,13457,2023-03-15


In [64]:
metro_am_sample = metro_am_sample >> mutate(no_seq = _.stop_sequence.isna())

In [65]:
metro_am_sample >> count(_.no_seq)

,no_seq,n
0,False,3912046


In [11]:
bbb_am_sample = get_sample_df(am_sampling_period)

* ~800mb for big blue bus AM sample alone, not enough RAM to sample all operators...
* LA Metro Bus barely fits, set nominal chunk size a bit lower by service hours...

In [12]:
bbb_am_sample

,arrival_time_pacific,departure_time_pacific,key,gtfs_dataset_key,base64_url,_extract_ts,trip_update_timestamp,trip_id,stop_sequence,stop_id,service_date
0,2023-03-15 08:20:08,NaT,e6401913e86dbbef5b610de6f4e82b2a,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:50:00+00:00,2023-03-15 14:48:00+00:00,894457,41,3,2023-03-15
1,2023-03-15 08:18:43,NaT,0af20483c80371f4cfc27cb581b097fd,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:38:20+00:00,2023-03-15 14:36:57+00:00,894457,41,3,2023-03-15
2,2023-03-15 08:18:08,NaT,8f052a8de87236cbdcef3ea0a5411600,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:22:20+00:00,2023-03-15 14:21:20+00:00,894457,41,3,2023-03-15
3,2023-03-15 08:22:12,NaT,965c2092c6d88c5762d22fb9eec477bb,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 15:01:00+00:00,2023-03-15 14:57:31+00:00,894457,41,3,2023-03-15
4,2023-03-15 08:21:16,NaT,55ad6f0c2b9b77c09edc13fb8f14a1ab,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 15:11:00+00:00,2023-03-15 15:08:08+00:00,894457,41,3,2023-03-15
...,...,...,...,...,...,...,...,...,...,...,...
368026,2023-03-15 08:00:24,NaT,8deabff571dbeaa6468410f968f61a93,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:04:40+00:00,2023-03-15 14:01:28+00:00,895148,37,1646,2023-03-15
368027,2023-03-15 08:00:45,NaT,0921c1733bafca364159ea9b1c8c4f9f,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:22:40+00:00,2023-03-15 14:21:27+00:00,895148,37,1646,2023-03-15
368028,2023-03-15 08:10:16,NaT,066d77b596286f157032bdaa6cbaa0c5,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 15:06:40+00:00,2023-03-15 15:05:23+00:00,895148,37,1646,2023-03-15
368029,2023-03-15 08:06:21,NaT,43cb9f4d04407afeb0cd5dc90dd4e12a,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:41:00+00:00,2023-03-15 14:38:30+00:00,895148,37,1646,2023-03-15


In [13]:
# ##TODO should group on feed and check if stop_seq populated, then apply; and can happen post download...

# def keep_first_stop(df):
#     df2 = (df >> group_by(_.trip_id)
#           >> filter(_.stop_sequence == _.stop_sequence.min())
#           >> ungroup()
#           )
#     return df2

In [14]:
keep_first_stop(bbb_am_sample)

,arrival_time_pacific,departure_time_pacific,key,gtfs_dataset_key,base64_url,_extract_ts,trip_update_timestamp,trip_id,stop_sequence,stop_id,service_date
2213,2023-03-15 08:25:46,2023-03-15 08:25:46,78f02525f9fe25b94b9dfc541375acd0,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:47:40+00:00,2023-03-15 14:45:37+00:00,894562,2,21,2023-03-15
2214,2023-03-15 08:25:46,2023-03-15 08:25:46,392cb7e79090d2d477daa2789ff1e934,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:51:20+00:00,2023-03-15 14:50:17+00:00,894562,2,21,2023-03-15
2215,2023-03-15 08:25:46,2023-03-15 08:25:46,4a7f78ff93b3acda8b926c25276f2728,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:39:40+00:00,2023-03-15 14:38:18+00:00,894562,2,21,2023-03-15
2216,2023-03-15 08:25:46,2023-03-15 08:25:46,fa815b0f502a9af105aba774e46d6be8,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:38:00+00:00,2023-03-15 14:35:06+00:00,894562,2,21,2023-03-15
2217,2023-03-15 08:25:46,2023-03-15 08:25:46,57cee5d59553a8dfe04218904968f6f5,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 15:15:20+00:00,2023-03-15 15:14:26+00:00,894562,2,21,2023-03-15
...,...,...,...,...,...,...,...,...,...,...,...
367508,2023-03-15 08:13:54,NaT,bb126bc28da8a05d727d86fad3b3a4ac,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:25:20+00:00,2023-03-15 14:24:09+00:00,895095,2,1640,2023-03-15
367509,2023-03-15 08:13:54,NaT,d34ca7dc38a9d984fec985537b97cd8b,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:39:00+00:00,2023-03-15 14:37:13+00:00,895095,2,1640,2023-03-15
367510,2023-03-15 08:13:54,NaT,d060388c8a77c93e1aad9c9c19deceb6,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:52:20+00:00,2023-03-15 14:50:22+00:00,895095,2,1640,2023-03-15
367511,2023-03-15 08:13:54,NaT,fe7ee9818be0db3f58503ef79ecc9589,44112e3a241100781ffa238f745ff442,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZG...,2023-03-15 14:49:40+00:00,2023-03-15 14:47:24+00:00,895095,2,1640,2023-03-15


In [17]:
# bbb_am_sample >> count(_.stop_sequence)

## select operators for chunking

In [13]:
analysis_ts = dt.datetime.combine(analysis_date, dt.time(0))

In [14]:
service_levels = (tbls.mart_gtfs.fct_daily_feed_scheduled_service_summary()
                  >> filter(_.activity_date == analysis_date)
                  >> select(_.activity_date, _.gtfs_dataset_key, _.ttl_service_hours)
)

In [15]:
tu_datasets = (tbls.mart_transit_database.dim_gtfs_datasets()
              >> filter(_.type == "trip_updates")
              >> select(_.trip_updates_gtfs_dataset_key == _.key,
                        _.tu_base64_url == _.base64_url)
              )

In [26]:
all_feeds = (tbls.mart_transit_database.dim_provider_gtfs_data()
    >> filter((_.regional_feed_type == None) | (_.regional_feed_type == 'Regional Subfeed'))
    >> filter(_._valid_from <= analysis_ts, _._valid_to > analysis_ts)
    >> filter(_.trip_updates_gtfs_dataset_name != None)
    >> select(_.organization_key, _.organization_name,
              _.regional_feed_type,
             _.associated_schedule_gtfs_dataset_key,
             _.schedule_gtfs_dataset_key, _.schedule_gtfs_dataset_name,
             _.trip_updates_gtfs_dataset_name, _.trip_updates_gtfs_dataset_key)
    >> inner_join(_, service_levels, on = {'associated_schedule_gtfs_dataset_key': 'gtfs_dataset_key'})
    >> inner_join(_, tu_datasets, on = 'trip_updates_gtfs_dataset_key')
    >> collect()
    >> distinct(_.trip_updates_gtfs_dataset_name, _keep_all=True)
    >> arrange(-_.ttl_service_hours)
)

In [27]:
all_feeds.columns

Index(['organization_key', 'organization_name', 'regional_feed_type',
       'associated_schedule_gtfs_dataset_key', 'schedule_gtfs_dataset_key',
       'schedule_gtfs_dataset_name', 'trip_updates_gtfs_dataset_name',
       'trip_updates_gtfs_dataset_key', 'activity_date', 'ttl_service_hours',
       'tu_base64_url'],
      dtype='object')

In [28]:
all_feeds.tu_base64_url.iloc[0]

'aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbGFtZXRyby9ndGZzLXJ0LXRyaXAtdXBkYXRlcw=='

In [29]:
all_feeds >> head(3)

,organization_key,organization_name,regional_feed_type,associated_schedule_gtfs_dataset_key,schedule_gtfs_dataset_key,schedule_gtfs_dataset_name,trip_updates_gtfs_dataset_name,trip_updates_gtfs_dataset_key,activity_date,ttl_service_hours,tu_base64_url
44,877f54da2be4733fbbc5a773b9b188d7,Los Angeles County Metropolitan Transportation...,None,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,LA Metro Bus Trip Updates,a163bb0928257e3654888c511f4b56dc,2023-03-15,17270.933333,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...
85,2519a1e3a5a76f9dac90fb8bb1ea7df5,City and County of San Francisco,Regional Subfeed,d8453d9a4ae1184f7e1339f7c61849df,d8453d9a4ae1184f7e1339f7c61849df,Bay Area 511 Muni Schedule,Bay Area 511 Muni TripUpdates,21fbd04001d3a2a21838b65a39d9088d,2023-03-15,6712.872222,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3RyaXB1cG...
74,637a87e4dd0240d002ea200f57b60245,San Diego Metropolitan Transit System,None,9707f17da050a601ceda7143d58fa9a2,9707f17da050a601ceda7143d58fa9a2,San Diego Schedule,San Diego Trip Updates,c1598c634dca9707a6262719ba85bff3,2023-03-15,5083.250000,aHR0cHM6Ly9yZWFsdGltZS5zZG10cy5jb20vYXBpL2FwaS...


In [35]:
import numpy as np

In [54]:
def chunk_by_svc_hours(df):
    
    chunks = {}
    # https://stackoverflow.com/questions/42763362/dividing-a-pandas-dataframe-into-smaller-chunks-based-of-the-sum-of-one-column
    chunk_target = 15 * 10**3 # slightly smaller than Metro Bus, which worked
    df['running_total'] = df['ttl_service_hours'].cumsum()
    # better to actually round than truncate as in stackoverflow example...
    df['batch'] = np.round((df['running_total'] / chunk_target), 0).astype(int)
    grouped = df.groupby('batch')
    for group in grouped.groups.keys():
        data = grouped.get_group(group)
        # start at 0, it's pythonic
        chunks[group - 1] =  data     
    return chunks

In [55]:
chunks = chunk_by_svc_hours(all_feeds)

In [56]:
chunks[0]

,organization_key,organization_name,regional_feed_type,associated_schedule_gtfs_dataset_key,schedule_gtfs_dataset_key,schedule_gtfs_dataset_name,trip_updates_gtfs_dataset_name,trip_updates_gtfs_dataset_key,activity_date,ttl_service_hours,tu_base64_url,running_total,batch
44,877f54da2be4733fbbc5a773b9b188d7,Los Angeles County Metropolitan Transportation...,None,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,LA Metro Bus Trip Updates,a163bb0928257e3654888c511f4b56dc,2023-03-15,17270.933333,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...,17270.933333,1


In [ ]:
## GCS format: st_updates_2023-03-15_am_sample

In [18]:
# shared_utils.rt_utils.show_full_df(all_feeds >> arrange(-_.ttl_service_hours))

,organization_key,organization_name,regional_feed_type,associated_schedule_gtfs_dataset_key,schedule_gtfs_dataset_key,schedule_gtfs_dataset_name,trip_updates_gtfs_dataset_name,trip_updates_gtfs_dataset_key,activity_date,ttl_service_hours,tu_base64_url
44,877f54da2be4733fbbc5a773b9b188d7,Los Angeles County Metropolitan Transportation...,None,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,LA Metro Bus Trip Updates,a163bb0928257e3654888c511f4b56dc,2023-03-15,17270.933333,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvbG...
85,2519a1e3a5a76f9dac90fb8bb1ea7df5,City and County of San Francisco,Regional Subfeed,d8453d9a4ae1184f7e1339f7c61849df,d8453d9a4ae1184f7e1339f7c61849df,Bay Area 511 Muni Schedule,Bay Area 511 Muni TripUpdates,21fbd04001d3a2a21838b65a39d9088d,2023-03-15,6712.872222,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3RyaXB1cG...
74,637a87e4dd0240d002ea200f57b60245,San Diego Metropolitan Transit System,None,9707f17da050a601ceda7143d58fa9a2,9707f17da050a601ceda7143d58fa9a2,San Diego Schedule,San Diego Trip Updates,c1598c634dca9707a6262719ba85bff3,2023-03-15,5083.250000,aHR0cHM6Ly9yZWFsdGltZS5zZG10cy5jb20vYXBpL2FwaS...
32,c73f84833af54e86518de137ae67ef3f,Alameda-Contra Costa Transit District,Regional Subfeed,444700afe086ed24e3cb888cecd3037c,444700afe086ed24e3cb888cecd3037c,Bay Area 511 AC Transit Schedule,Bay Area 511 AC Transit Trip Updates,6aba5c314d297bcedd7ec86fcaf9361c,2023-03-15,4014.133333,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3RyaXB1cG...
1,9f75f5594cf33f8581b3668c7af41ea8,Santa Clara Valley Transportation Authority,Regional Subfeed,a065f1788f6694f048a3908e0adb1b57,a065f1788f6694f048a3908e0adb1b57,Bay Area 511 Santa Clara Transit Schedule,Bay Area 511 Santa Clara Transit TripUpdates,a53b8a92a2d726f3ecdd89e3d7b23e41,2023-03-15,3704.566667,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L3RyaXB1cG...
6,896b08fb0bb4343b7d03ed8c46db7aa8,Orange County Transportation Authority,None,f3cd87af530fa12c8b6d355697f8d38a,f3cd87af530fa12c8b6d355697f8d38a,OCTA Schedule,OCTA TripUpdates,bcaf0e7ba3d6894cde5195ceea3f7d7e,2023-03-15,3648.883889,aHR0cHM6Ly9hcGkuZ29zd2lmdC5seS9yZWFsLXRpbWUvb2...
79,fc50ede96355f6d9b575d351ee5d065a,City of Los Angeles,None,08862f6780e2bb5afce698a73f6974cc,08862f6780e2bb5afce698a73f6974cc,LA DOT Schedule,LA DOT TripUpdates,b115caeb76c5b11134649b746958935e,2023-03-15,2382.683333,aHR0cHM6Ly9sYWRvdGJ1cy5jb20vZ3Rmcy1ydC90cmlwdX...
51,6450980a2aca9da5bad1c6b7e328cb93,City of Duarte,None,f74424acf8c41e4c1e9fd42838c4875c,f74424acf8c41e4c1e9fd42838c4875c,Foothill Schedule,Foothill Trip Updates,e1d7189626caa35fa333f4a5df0674d7,2023-03-15,2325.266667,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...
8,29aac8a3357294493304dc8beefc4d9e,Anaheim Transportation Network,None,b7a6cd6a1a06406c35fa9abd16ad9754,b7a6cd6a1a06406c35fa9abd16ad9754,Anaheim Resort Schedule,Anaheim Resort TripUpdates,262d7b27183fa8d174ab8fc83ad5848f,2023-03-15,1936.420833,aHR0cHM6Ly9hcnQudHJpcHNob3QuY29tL3YxL2d0ZnMvcm...
88,797e95cb8e592aa52fef209773f2df0c,Long Beach Transit,None,32230adf5db3a8df91f05947bb1ad658,32230adf5db3a8df91f05947bb1ad658,Long Beach Schedule,LongBeach TripUpdates,92a4eb490b324bc0e6212bedd81f94cb,2023-03-15,1613.976111,aHR0cHM6Ly9sYnRndGZzLmxidHJhbnNpdC5jb20vVE1HVE...


## schedule sandbox, match example

In [37]:
import pytz

In [40]:
pacific = pytz.timezone('US/Pacific')

In [3]:
bbb_ix_df = shared_utils.rt_utils.get_speedmaps_ix_df(analysis_date, 300)

In [4]:
bbb_ix_df

,feed_key,schedule_gtfs_dataset_key,vehicle_positions_gtfs_dataset_key,organization_itp_id,organization_name,activity_date
0,4f77ef02b983eccc0869c7540f98a7d0,dbbe8ee4864a2715a40749605395d584,6c2d7daaf979779fa2089c6395baf98b,300,City of Santa Monica,2023-03-15


In [46]:
bbb_trips = shared_utils.rt_utils.get_trips(bbb_ix_df)

found trips parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/trips_300_2023-03-15.parquet


In [49]:
bbb_st = shared_utils.rt_utils.get_st(bbb_ix_df, bbb_trips)

found stop times parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/st_300_2023-03-15.parquet


In [57]:
bbb_st.arrival_time.iloc[0].split(':')

['06', '26', '02']

In [61]:
shared_utils.rt_utils.show_full_df(bbb_st >> group_by(_.trip_id) >> summarize(min_arrival = _.arrival_time.min()))

,trip_id,min_arrival
0,893044,06:59:00
1,893045,07:10:00
2,893046,07:19:00
3,893047,07:29:00
4,893048,07:40:00
5,893049,07:49:00
6,893050,07:59:00
7,893051,08:10:00
8,893052,08:20:00
9,893053,08:30:00


In [25]:
# df = (
#     tbls.mart_ad_hoc.fct_stop_time_updates_20230315_to_20230321()
#     >> filter(_.service_date == analysis_date,
#               # _.trip_id == '894836',
#               _.arrival_time_pacific >= pac_ts,
#               _.arrival_time_pacific < pac_ts2,
#              _.base64_url == bbb_base64)
#     >> select(_.arrival_time_pacific, _.departure_time_pacific,
#              _.key, _.gtfs_dataset_key, _.base64_url,
#               _._extract_ts, _.trip_update_timestamp, _.trip_id,
#              _.stop_sequence, _.stop_id, _.service_date)
#     >> collect()
# )

In [12]:
df.base64_url.iloc[0]

'aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZGF0ZXMuYmlu'

In [21]:
df.columns

Index(['key', 'gtfs_dataset_key', 'dt', 'hour', 'base64_url', '_extract_ts',
       '_config_extract_ts', '_gtfs_dataset_name', '_header_message_age',
       '_trip_update_message_age', '_trip_update_message_age_vs_header',
       'header_timestamp', 'header_version', 'header_incrementality', 'id',
       'trip_update_timestamp', 'trip_update_delay', 'vehicle_id',
       'vehicle_label', 'vehicle_license_plate', 'trip_id', 'trip_route_id',
       'trip_direction_id', 'trip_start_time', 'trip_start_date',
       'trip_schedule_relationship', '_trip_updates_message_key',
       'stop_sequence', 'stop_id', 'arrival_delay', 'arrival_time',
       'arrival_uncertainty', 'departure_delay', 'departure_time',
       'departure_uncertainty', 'schedule_relationship', 'service_date',
       'arrival_time_pacific', 'departure_time_pacific'],
      dtype='object')

In [24]:
df.to_parquet('bbb_example.parquet')

In [29]:
df.stop_sequence.isna().value_counts()

False    684351
Name: stop_sequence, dtype: int64

In [36]:
filtered._extract_ts.iloc[0]

Timestamp('2023-03-15 14:43:20+0000', tz='UTC')

In [62]:
filtered = df >> filter(_.trip_id == '893051', _.stop_sequence == 2) >> arrange(_._extract_ts)
filtered = filtered >> mutate(arrival_time_pacific = _.arrival_time_pacific.apply(lambda x: pacific.localize(x)))
filtered = filtered >> mutate(time_in_advance = _.arrival_time_pacific - _._extract_ts)
filtered = filtered >> select(-_.key, -_.gtfs_dataset_key, -_.base64_url)

In [ ]:
## yep, this is what we wanna see!
## can do a tighter arrival time filter =-- 30min periods??

In [65]:
shared_utils.rt_utils.show_full_df(filtered)

,arrival_time_pacific,departure_time_pacific,_extract_ts,trip_update_timestamp,trip_id,stop_sequence,stop_id,service_date,time_in_advance
276780,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:10:40+00:00,2023-03-15 14:10:30+00:00,893051,2,361,2023-03-15,0 days 01:00:14
277242,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:11:00+00:00,2023-03-15 14:10:50+00:00,893051,2,361,2023-03-15,0 days 00:59:54
277661,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:11:20+00:00,2023-03-15 14:11:06+00:00,893051,2,361,2023-03-15,0 days 00:59:34
275258,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:11:40+00:00,2023-03-15 14:11:30+00:00,893051,2,361,2023-03-15,0 days 00:59:14
275680,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:12:00+00:00,2023-03-15 14:11:50+00:00,893051,2,361,2023-03-15,0 days 00:58:54
276279,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:12:20+00:00,2023-03-15 14:12:06+00:00,893051,2,361,2023-03-15,0 days 00:58:34
276960,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:12:40+00:00,2023-03-15 14:12:30+00:00,893051,2,361,2023-03-15,0 days 00:58:14
277484,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:13:00+00:00,2023-03-15 14:12:50+00:00,893051,2,361,2023-03-15,0 days 00:57:54
278639,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:13:20+00:00,2023-03-15 14:13:11+00:00,893051,2,361,2023-03-15,0 days 00:57:34
275418,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:13:40+00:00,2023-03-15 14:13:30+00:00,893051,2,361,2023-03-15,0 days 00:57:14


In [ ]:
## sampling approach: use 8a, noon, 11p for all operators?

In [9]:
pac_ts = dt.datetime.combine(analysis_date, dt.time(8, 0))

In [14]:
pac_ts_sec = int(pac_ts.timestamp())

In [15]:
pac_ts2 = pac_ts + dt.timedelta(hours = 1)

In [16]:
pac_ts2_sec = int(pac_ts2.timestamp())